# MLOps

![MLOps](images/MLOps.jpg)

![hidden technical debt paper](images/hidden_technical_debt_2015.jpg)

Heute im Fokus
* __Reproduzierbarkeit__: 
    * Versionierung von Daten und Code
* __Bereitstellung__: 
    * Modell API
* __Monitoring__:
    * Überwachung des Verhaltens des Modells in Produktion

# Code

* GitHub: https://github.com/datanizing/datascienceday/
* Verzeichnis: `06_MLOps`

# Modell aus "Sprachmodelle und Sentiment-Analyse" (Oliver Zeigermann)
![Architektur Überblick](images/Architecture_Python.png)

# Reproduzierbarkeit mit DVC
![Architektur Überblick](images/Architecture_Minio.png)

## [DVC](https://dvc.org/)

![DVC_project_versions](https://dvc.org/static/39d86590fa8ead1cd1247c883a8cf2c0/fa73e/project-versions.webp)

## DVC Pipelines
![DVC Pipeline Example](https://dagshub.com/docs/tutorial/assets/process_and_train_repo.png)
Quelle: https://dagshub.com/

## DVC Pipelines

#### Daten laden

```
python load_data.py
```

In [18]:
!dvc run -n load_data --force -o data/raw/transport-short.csv -d load_data.py python load_data.py

Stage 'load_data' is cached - skipping run, checking out outputs      core>
Modifying stage 'load_data' in 'dvc.yaml'                                       

To track the changes with git, run:

    git add dvc.yaml

To enable auto staging, run:

	dvc config core.autostage true


## DVC Pipelines

#### Model trainieren

```
python train.py
```

In [19]:
!dvc run -n train --force -d data/raw/transport-short.csv -d train.py -o models/model/ python train.py

Stage 'train' is cached - skipping run, checking out outputs          core>
Modifying stage 'train' in 'dvc.yaml'                                           

To track the changes with git, run:

    git add dvc.yaml

To enable auto staging, run:

	dvc config core.autostage true


## DVC Pipelines

In [20]:
!dvc dag --full | cat

+-----------+  
| load_data |  
+-----------+  
      *        
      *        
      *        
  +-------+    
  | train |    
  +-------+    


In [21]:
!dvc dag --out | cat

+------------------------------+ 
| data/raw/transport-short.csv | 
+------------------------------+ 
                *                
                *                
                *                
        +--------------+         
        | models/model |         
        +--------------+         


## Pipelines
Mit DVC lassen sich Pipelines definieren um die komplette Pipeline von den Rohdaten bis zum Modell reprouzierbar zu machen.
#### `dvc.yaml`

```yaml
stages:
  train:
    cmd: python train.py
    deps:
    - data/raw/transport-short.csv
    outs:
    - models/model/

```

Reproduzieren der Schritte mit

In [22]:
!dvc repro

Stage 'load_data' didn't change, skipping                             core>
Stage 'train' didn't change, skipping
Data and pipelines are up to date.


## Datenstände

Datenstände werden in `dvc.lock` über Hashes abgebildet.
```yaml
stages:
  load_data:
    cmd: python load_data.py
    deps:
    - path: load_data.py
      md5: ddeb3c7968c47788fb055752566e725d
      size: 153
    outs:
    - path: data/raw/transport-short.csv
      md5: 3057d4f316405b0a282328d2f9ee5748
      size: 551260620
  train:
     ...
```

## DVC Data

Folgende Datenablagen werden unterstützt:
* Amazon S3 (und kompatible, z.B. Minio)
* Azure Blob Storage
* Google Drive
* Google Cloud Storage
* Aliyon OSS
* SSH
* HDFS
* WebHDFS
* HTTP
* WebDAV
* local remote (z.B. Netzlaufwerke)

## DVC Remote

Hier, [DVC Remote with Minio](http://localhost:9000/minio/dvcrepo/)

* ACCESS KEY: `minio-access-key`
* SECRET KEY: `minio-secret-key`

In [23]:
!dvc push

Everything is up to date.                                                       


In [24]:
!dvc pull

Everything is up to date.                                                       


# Docker Image mit API
![Architektur Überblick](images/Architecture_Docker.png)

# Sentiment API

Mittels [FastAPI](https://fastapi.tiangolo.com/) wird eine API für das Sentiment Model bereitgestellt.

### Datenmodell für Ein- und Ausgabe

##### `app.py`
```python
from pydantic import BaseModel, Field

# Datenmodell der Eingabe
class Input(BaseModel):
    sentence: str = Field(example="Das ist ein toller Satz.")

# Datenmodell der Ausgabe
class Sentiment(BaseModel):
    label: str = Field(description="Sentiment", example="NEGATIVE")
    score: float = Field(description="Score", example=0.9526780247688293)
```

### API Endpunkt

##### `app.py`
```python
from fastapi import FastAPI, Response

# Erzeugen der FastAPI Anwendung
app = FastAPI(
    title="Sentiment Model API",
    description="Sentiment Model API",
    version="0.1",)
```


###### `app.py`
```python
# Modell laden
sentiment_classifier = pipeline("sentiment-analysis", "models/model")

@app.post('/predict', response_model=Sentiment, operation_id="predict_post")
async def predict(response: Response, input: Input):
        pred=sentiment_classifier(input.sentence)[0]
        sentiment = Sentiment.parse_obj(pred)
        return sentiment
```

## API Testen

[uvicorn Webserver](https://www.uvicorn.org/)

##### `app.py`
```python
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8080)
```

Webserver starten
```
python app.py
```

## Schnittstelle

FastAPI stellt eine Dokumentation der Schnittstelle unter [/docs](http://localhost:8080/docs) zur Verfügung.

### Client Code erzeugen
Der Client wird mittles [`openapi-python-client` Generator](https://github.com/openapi-generators/openapi-python-client) z.B. wie folgt erzeugt.
```
openapi-python-client generate --url http://127.0.0.1/openapi.json
```

In [26]:
!openapi-python-client generate --url http://127.0.0.1:8080/openapi.json

Generating sentiment-model-api-client
Error(s) encountered while generating, client was not created

Unable to generate the client

Directory already exists. Delete it or use the update command.


If you believe this was a mistake or this tool is missing a feature you need, please open an issue at https://github.com/openapi-generators/openapi-python-client/issues/new/choose


[Andere Client-Generatoren](https://openapi-generator.tech/docs/generators/#client-generators)

## API aufrufen

In [27]:
import sys
sys.path.append("./sentiment-model-api-client")
from sentiment_model_api_client.client import Client
from sentiment_model_api_client.models import Input
from sentiment_model_api_client.api.default import predict_post

client = Client(base_url="http://localhost:8080", timeout=30)

predict_post.sync(client=client, 
                  json_body=Input(sentence=
                                  "I wonder how close a drone has to get to private property before someone "
                                  "can shoot it down, because that will definitely happen."))

Sentiment(label='POSITIVE', score=0.9907467365264893, additional_properties={})

## Docker Images bauen

##### `Dockerfile` (Auszug)
```Dockerfile
FROM docker.io/bitnami/python:3.8.13
...
RUN pip install -r requirements.txt --no-cache-dir 

...
RUN dvc config core.no_scm true && \
    dvc pull models/model/

CMD uvicorn app:app --host=0.0.0.0 --port=8080
```

In [28]:
!docker build -t modelapi --network="host" .

Sending build context to Docker daemon  1.642GB
Step 1/15 : FROM docker.io/bitnami/python:3.8.13
 ---> ce5e98c6424c
Step 2/15 : EXPOSE 8080
 ---> Using cache
 ---> 31b3be169fdb
Step 3/15 : USER root
 ---> Using cache
 ---> e6e5f96b6374
Step 4/15 : WORKDIR /app
 ---> Using cache
 ---> 20b4ea5fe3df
Step 5/15 : COPY requirements.txt .
 ---> Using cache
 ---> a248a5664d55
Step 6/15 : RUN pip install -r requirements.txt --no-cache-dir
 ---> Using cache
 ---> 8bd239d599da
Step 7/15 : COPY dvc.yaml dvc.yaml
 ---> Using cache
 ---> a680bbb27aa6
Step 8/15 : COPY dvc.lock dvc.lock
 ---> Using cache
 ---> 818547c2e2e8
Step 9/15 : COPY .dvc/config .dvc/config
 ---> Using cache
 ---> aea27fb4bb7a
Step 10/15 : RUN dvc config core.no_scm true &&     dvc pull models/model/
 ---> Using cache
 ---> 43b9fecf376d
Step 11/15 : COPY app.py .
 ---> Using cache
 ---> 8425caeed581
Step 12/15 : RUN chgrp -R 0 . &&     chmod -R g=u . &&     chmod -R g+rw . &&     chmod a+x app.py
 ---> Using cache
 ---> 990b22da

# Orchestrierung mit Docker
![Architektur Überblick](images/Architecture_DockerCompose.png)

## Docker-Compose Datei
```yaml
version: "2"
services:
    modelapi:
        image: modelapi
        expose:
        - 8080
        ports:
        - 8080:8080

    minio:
        image: docker.io/bitnami/minio:2021.6.17
        ...
        
    prometheus:
        image: docker.io/bitnami/prometheus:2
        ...

    grafana:
        image: docker.io/bitnami/grafana:7
        ...
```


Starten mit

```
docker-compose up
```

# Monitoring mit Prometheus/Grafana
![Architektur Überblick](images/Architecture_Grafana.png)

# Monitoring

Modelausgaben über Header mitgeben, damit der Prometheus Client diese abfangen kann.

#### `app.py`
```python
# Endpunkt für Prediction
@app.post('/predict', response_model=Sentiment, operation_id="predict_post")
async def predict(response: Response, input: Input):
    pred = sentiment_classifier(input.sentence)[0]
    sentiment = Sentiment(**pred)

    # Header Monitoring
    response.headers["X-model-score"] = str(sentiment.score)
    response.headers["X-model-sentiment"] = str(sentiment.label)

    return sentiment


Metriken definieren

```python
from prometheus_client import Histogram, Counter

def model_output(metric_namespace: str = "", metric_subsystem: str = ""):
    SCORE = Histogram(
        "model_score",
        "Predicted score of model",
        buckets=(0, .1, .2, .3, .4, .5, .6, .7, .8, .9),
        namespace="mlops",
        subsystem="model",
    )
    ...
```

```python
    ...
    SENTIMENT = Counter(
        "sentiment",
        "Predicted sentiment",
        namespace="mlops",
        subsystem="model",
        labelnames=("sentiment",)        
    )
    ...
```

Metriken aus Header auslesen

```python
    ...
    def instrumentation(info) -> None:
        if info.modified_handler == "/predict":
            model_score = info.response.headers.get("X-model-score")
            model_sentiment = info.response.headers.get("X-model-sentiment")
            if model_score:
                SCORE.observe(float(model_score))
                SENTIMENT.labels(model_sentiment).inc()

    return instrumentation
```

Metriken an `app` beobachten:
```python
from prometheus_fastapi_instrumentator import Instrumentator

instrumentator = Instrumentator()
instrumentator.add(model_output())

# Prometheus Instrumentator verknüpfen
instrumentator.instrument(app).expose(app)
```

# Monitoring

[Model API](http://localhost:8080/docs)

[Metrics Endpoint](http://localhost:8080/metrics)

[Prometheus](http://localhost:9090/graph?g0.expr=mlops_model_model_score_bucket%20&g0.tab=1&g0.stacked=0&g0.show_exemplars=0&g0.range_input=1h)

[Grafana](http://localhost:3000)

## 500 Aufrufe

#### Daten laden

In [ ]:
import pandas as pd
data_df = pd.read_csv("data/raw/transport-short.csv", header=None, nrows=1000, names=['id', 'kind', 'title', 'link_id', 'parent_id', 'ups', 'downs', 'score',
       'author', 'num_comments', 'created_utc', 'permalink', 'url', 'text',
       'level', 'top_parent'])
data_df.head(2)

### API mit 500 Samples aufrufen

In [ ]:
for idx, row in data_df.sample(500).iterrows():
    if not pd.isna(row["text"]) and row["text"] not in ["[deleted]", "[removed]"]:
        sentiment = predict_post.sync(client=client, json_body=Input(sentence=row["text"]))
        data_df.loc[idx, "sentiment"] = sentiment.label

Ergebnisse

In [ ]:
with pd.option_context("display.max_colwidth", None):
    display(data_df[pd.notna(data_df["sentiment"])][["text", "sentiment"]].head())

# [Dashboard](http://localhost:3000/d/PGUZYQznk/model-score?orgId=1&refresh=5s)

![dashboard showing distriubtions of models scores, outlier scores, labels and drifts over time](images/dashboard.png)



## Was kann man messen?

* Score Verteilung
   * Frühwarnindikator für Probleme
   * Leichter zu messen als Modellgüte ("Was ist das korrekte Sentiment?")
* Grundlegende Aufrufstatistiken
   * Wird das Modell ggf. anders verwendet?
   
Komplexer, aber ggf. hilfreich:
* Drift
   * Seperates Modell notwendig ("Drift Detector")
   * z.B. Themenschwerpunkte verschieben sich im Vergleich zum Trainingsdatensatz stark
* Outlier Score
   * Separate Modelle, die prüfen, ob Daten zu Trainingsdaten passen
   * z.B. Anteil nicht-englischer Posts steigt